# Annotations Implementation

## Setup

In [1]:
import os
import sys

main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(main_dir)
print(main_dir)

from MambaFCS.changedetection.configs.config import get_config

import torch
import numpy as np
import imageio
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import functional as TF
import torch.nn.functional as F
import torch_optimizer as topt
from tqdm import tqdm
from MambaFCS.changedetection.datasets.make_data_loader import ChangeDetectionDatset, make_data_loader
from MambaFCS.changedetection.utils_func.metrics import Evaluator
from MambaFCS.changedetection.models.MambaBCD import STMambaBCD
import MambaFCS.changedetection.utils_func.lovasz_loss as L
from MambaFCS.changedetection.utils_func.loss import ce2_dice1
import MambaFCS.changedetection.datasets.imutils as imutils

from dotenv import load_dotenv

load_dotenv()
def getPath(env_path):
    return os.path.expanduser(os.getenv(env_path))


torch.cuda.set_device(1)

/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/classification/models/vmamba.py:252: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/classification/models/vmamba.py:260: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/classification/models/vmamba.py:275: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is 

In [6]:
configs_path = os.path.join(main_dir, 'MambaFCS/changedetection/configs/vssm1/vssm_base_224.yaml')

model_path_SYSU = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/MambaBCD_SYSU_base/12500_model_0.8151406739201706.pth')
model_path_LEVIR = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/MambaBCD_LEVIR_base/32500_model_0.9259150291513656.pth')
model_path_WHU = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/MambaBCD_WHU_base/20000_model_0.9389692423043274.pth')

LEVIR_dataset_path = getPath('LEVIRCDPATH')

LEVIR_train_dataset_path = os.path.join(LEVIR_dataset_path, 'train')
LEVIR_test_dataset_path = os.path.join(LEVIR_dataset_path, 'test')
LEVIR_train_data_list_path = os.path.join(LEVIR_dataset_path, 'train.txt')
LEVIR_test_data_list_path = os.path.join(LEVIR_dataset_path, 'test.txt')

SYSU_dataset_path = getPath('SYSUCDPATH')

SYSU_train_dataset_path = os.path.join(SYSU_dataset_path, 'train')
SYSU_test_dataset_path = os.path.join(SYSU_dataset_path, 'test')
SYSU_train_data_list_path = os.path.join(SYSU_dataset_path, 'train.txt')
SYSU_test_data_list_path = os.path.join(SYSU_dataset_path, 'test.txt')

WHU_dataset_path = getPath('WHUCDPATH')

WHU_train_dataset_path = os.path.join(WHU_dataset_path, 'train')
WHU_test_dataset_path = os.path.join(WHU_dataset_path, 'test')
WHU_train_data_list_path = os.path.join(WHU_dataset_path, 'train.txt')
WHU_test_data_list_path = os.path.join(WHU_dataset_path, 'test.txt')

VSSM_MODEL_PATH = getPath('VSSMBASEPATH')
model_path = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/')

SAVE_IMGS_PATH = os.path.join(main_dir, 'MambaFCS/annotations/Images/BCD/')

In [7]:
train_data_list = []
with open(LEVIR_train_data_list_path, 'r') as f:
    for line in f:
        train_data_list.append(line.strip())

test_data_list = []
with open(LEVIR_test_data_list_path, 'r') as f:
    for line in f:
        test_data_list.append(line.strip())

class ARGS:
    def __init__(self):
        self.dataset_path = LEVIR_dataset_path
        self.pretrained_weight_path = VSSM_MODEL_PATH
        self.dataset = 'LEVIR-CD+'
        self.opts = None
        self.type = 'train'
        self.shuffle = True
        self.crop_size = 256
        self.batch_size = 4
        self.max_iters = 1600000
        self.start_iter = 0
        self.cuda = True
        self.model_type = 'MambaBCD'
        self.learning_rate = 1e-4
        self.momentum = 0.9
        self.weight_decay = 5e-4
        self.train_dataset_path = LEVIR_train_dataset_path
        self.train_data_name_list = train_data_list
        self.test_dataset_path = LEVIR_test_dataset_path
        self.test_data_name_list = test_data_list
        self.cfg = configs_path
        self.model_param_path = model_path
        self.resume = model_path_LEVIR

        self.optimizer = 'adamw'

In [8]:
def LEVIR_main():
    args = ARGS()
    return args

def SYSU_main():
    args = ARGS()
    args.dataset = 'SYSU'
    args.resume = model_path_SYSU
    args.model_saving_name = 'MambaBCD_SYSU_base'
    args.dataset_path = SYSU_dataset_path
    args.train_dataset_path = SYSU_train_dataset_path
    args.train_data_name_list = []
    with open(SYSU_train_data_list_path, 'r') as f:
        for line in f:
            args.train_data_name_list.append(line.strip())
    args.test_dataset_path = SYSU_test_dataset_path
    args.test_data_name_list = []
    with open(SYSU_test_data_list_path, 'r') as f:
        for line in f:
            args.test_data_name_list.append(line.strip())
    
    return args

def WHU_main():
    args = ARGS()
    args.dataset = 'WHU-CD'
    args.resume = model_path_WHU
    args.model_saving_name = 'MambaBCD_WHU_base'
    args.dataset_path = WHU_dataset_path
    args.train_dataset_path = WHU_train_dataset_path
    args.train_data_name_list = []
    with open(WHU_train_data_list_path, 'r') as f:
        for line in f:
            args.train_data_name_list.append(line.strip())
    args.test_dataset_path = WHU_test_dataset_path
    args.test_data_name_list = []
    with open(WHU_test_data_list_path, 'r') as f:
        for line in f:
            args.test_data_name_list.append(line.strip())
    
    return args

In [21]:
args = SYSU_main()
config = get_config(args)

train_data_loader = make_data_loader(args)

evaluator = Evaluator(num_class=2)

deep_model = STMambaBCD(
    pretrained=args.pretrained_weight_path,
    patch_size=config.MODEL.VSSM.PATCH_SIZE, 
    in_chans=config.MODEL.VSSM.IN_CHANS, 
    num_classes=config.MODEL.NUM_CLASSES, 
    depths=config.MODEL.VSSM.DEPTHS, 
    dims=config.MODEL.VSSM.EMBED_DIM, 
    # ===================
    ssm_d_state=config.MODEL.VSSM.SSM_D_STATE,
    ssm_ratio=config.MODEL.VSSM.SSM_RATIO,
    ssm_rank_ratio=config.MODEL.VSSM.SSM_RANK_RATIO,
    ssm_dt_rank=("auto" if config.MODEL.VSSM.SSM_DT_RANK == "auto" else int(config.MODEL.VSSM.SSM_DT_RANK)),
    ssm_act_layer=config.MODEL.VSSM.SSM_ACT_LAYER,
    ssm_conv=config.MODEL.VSSM.SSM_CONV,
    ssm_conv_bias=config.MODEL.VSSM.SSM_CONV_BIAS,
    ssm_drop_rate=config.MODEL.VSSM.SSM_DROP_RATE,
    ssm_init=config.MODEL.VSSM.SSM_INIT,
    forward_type=config.MODEL.VSSM.SSM_FORWARDTYPE,
    # ===================
    mlp_ratio=config.MODEL.VSSM.MLP_RATIO,
    mlp_act_layer=config.MODEL.VSSM.MLP_ACT_LAYER,
    mlp_drop_rate=config.MODEL.VSSM.MLP_DROP_RATE,
    # ===================
    drop_path_rate=config.MODEL.DROP_PATH_RATE,
    patch_norm=config.MODEL.VSSM.PATCH_NORM,
    norm_layer=config.MODEL.VSSM.NORM_LAYER,
    downsample_version=config.MODEL.VSSM.DOWNSAMPLE,
    patchembed_version=config.MODEL.VSSM.PATCHEMBED,
    gmlp=config.MODEL.VSSM.GMLP,
    use_checkpoint=config.TRAIN.USE_CHECKPOINT,
    ) 

checkpoint = torch.load(args.resume)
model_dict = {}
state_dict = deep_model.state_dict()
for k, v in checkpoint.items():
    if k in state_dict:
        model_dict[k] = v
state_dict.update(model_dict)
deep_model.load_state_dict(state_dict)


deep_model.cuda()
deep_model.eval()

=> merge config from /storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/changedetection/configs/vssm1/vssm_base_224.yaml
Successfully load ckpt /storage/scratch3/buddhiw-change-detection/ChangeDetection/VSSModels/pretrained/vssm_base_0229_ckpt_epoch_237.pth
_IncompatibleKeys(missing_keys=['outnorm0.weight', 'outnorm0.bias', 'outnorm1.weight', 'outnorm1.bias', 'outnorm2.weight', 'outnorm2.bias', 'outnorm3.weight', 'outnorm3.bias'], unexpected_keys=['classifier.norm.weight', 'classifier.norm.bias', 'classifier.head.weight', 'classifier.head.bias'])


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


STMambaBCD(
  (encoder): Backbone_VSSM(
    (patch_embed): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): Permute()
      (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (3): Permute()
      (4): GELU(approximate='none')
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (6): Permute()
      (7): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (layers): ModuleList(
      (0): Sequential(
        (blocks): Sequential(
          (0): VSSBlock(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (op): SS2D(
              (out_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (in_proj): Linear(in_features=128, out_features=256, bias=False)
              (act): SiLU()
              (conv2d): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=256, bias=False)
              (out_proj): Linear(in_

In [22]:
def img_loader(path):
    img = np.array(imageio.imread(path), np.float32)
    return img

# Ground Truth Annotations

In [23]:
def annotate_image(image_number, dataset):

    if dataset == 'LEVIR-CD+':
        path = LEVIR_test_dataset_path
    elif dataset == 'SYSU':
        path = SYSU_test_dataset_path
    elif dataset == 'WHU-CD':
        path = WHU_test_dataset_path

    pre_image_path = os.path.join(path,"T1",f"{image_number}.png")
    post_image_path = os.path.join(path,"T2",f"{image_number}.png")
    GT_image_path = os.path.join(path,"GT",f"{image_number}.png")

    pre_image = img_loader(pre_image_path)
    post_image = img_loader(post_image_path)
    GT_image = img_loader(GT_image_path)

    pre_img = imutils.normalize_img(pre_image)  # imagenet normalization
    pre_img = np.transpose(pre_img, (2, 0, 1))

    post_img = imutils.normalize_img(post_image)  # imagenet normalization
    post_img = np.transpose(post_img, (2, 0, 1))

    GT_img = GT_image/255

    # save original images
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pre.png"), pre_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_post.png"), post_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_gt.png"), GT_image.astype(np.uint8))

    pre_tensor = TF.to_tensor(imutils.normalize_img(pre_image)).unsqueeze(0).cuda()
    post_tensor = TF.to_tensor(imutils.normalize_img(post_image)).unsqueeze(0).cuda()
    label_tensor = torch.tensor(GT_image / 255.0).unsqueeze(0).cuda()

    with torch.no_grad():
        output = deep_model(pre_tensor, post_tensor)
        output = output.data.cpu().numpy()
        output = np.argmax(output, axis=1)

    H, W = output[0].shape
    overlay = np.zeros((H, W, 3), dtype=np.uint8)   # start black

    white_mask = (output[0] == 1)
    overlay[white_mask] = [255, 255, 255] 

    # True Negatives: output==0, gt==0  → RED
    tn_mask = (output[0] == 0) & (GT_img == 1)
    overlay[tn_mask] = [255, 0, 0]

    # False Positives: output==1, gt==0 → GREEN
    fp_mask = (output[0] == 1) & (GT_img == 0)
    overlay[fp_mask] = [0, 255, 0]

    # ------------------------------------------------------------------
    # Option-1: save just the coloured mask
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_TN_FP.png"), overlay)

# WHUCD

In [24]:
images = [
    "whucd_00001", "whucd_01163", "whucd_02380", "whucd_03840", "whucd_05017", "whucd_06165",
    "whucd_00003", "whucd_01167", "whucd_02384", "whucd_03849", "whucd_05022", "whucd_06202",
    "whucd_00016", "whucd_01179", "whucd_02386", "whucd_03868", "whucd_05026", "whucd_06210",
    "whucd_00023", "whucd_01195", "whucd_02416", "whucd_03890", "whucd_05030", "whucd_06213",
    "whucd_00027", "whucd_01207", "whucd_02421", "whucd_03895", "whucd_05037", "whucd_06229",
    "whucd_00031", "whucd_01210", "whucd_02422", "whucd_03909", "whucd_05043", "whucd_06239",
    "whucd_00033", "whucd_01211", "whucd_02434", "whucd_03910", "whucd_05061", "whucd_06244",
    "whucd_00055", "whucd_01212", "whucd_02435", "whucd_03917", "whucd_05078", "whucd_06247",
    "whucd_00057", "whucd_01216", "whucd_02439", "whucd_03952", "whucd_05080", "whucd_06249",
    "whucd_00059", "whucd_01224", "whucd_02440", "whucd_03968", "whucd_05081", "whucd_06250",
    "whucd_00073", "whucd_01230", "whucd_02443", "whucd_03975", "whucd_05090", "whucd_06276",
    "whucd_00087", "whucd_01241", "whucd_02446", "whucd_03980", "whucd_05092", "whucd_06284",
    "whucd_00089", "whucd_01243", "whucd_02458", "whucd_03985", "whucd_05098", "whucd_06295",
    "whucd_00091", "whucd_01245", "whucd_02459", "whucd_04004", "whucd_05104", "whucd_06318",
    "whucd_00092", "whucd_01248", "whucd_02473", "whucd_04018", "whucd_05150", "whucd_06321",
    "whucd_00094", "whucd_01253", "whucd_02478", "whucd_04030", "whucd_05154", "whucd_06326",
    "whucd_00099", "whucd_01254", "whucd_02481", "whucd_04037", "whucd_05167", "whucd_06337",
    "whucd_00129", "whucd_01258", "whucd_02489", "whucd_04038", "whucd_05171", "whucd_06339",
    "whucd_00138", "whucd_01287", "whucd_02495", "whucd_04048", "whucd_05187", "whucd_06340",
    "whucd_00144", "whucd_01309", "whucd_02496", "whucd_04057", "whucd_05202", "whucd_06347",
    "whucd_00148", "whucd_01315", "whucd_02538", "whucd_04071", "whucd_05222", "whucd_06362",
    "whucd_00156", "whucd_01320", "whucd_02543", "whucd_04080", "whucd_05233", "whucd_06365",
    "whucd_00159", "whucd_01342", "whucd_02562", "whucd_04089", "whucd_05238", "whucd_06386",
    "whucd_00170", "whucd_01386", "whucd_02565", "whucd_04090", "whucd_05251", "whucd_06395",
    "whucd_00176", "whucd_01390", "whucd_02572", "whucd_04097", "whucd_05264", "whucd_06417",
    "whucd_00178", "whucd_01395", "whucd_02578", "whucd_04107", "whucd_05269", "whucd_06454",
    "whucd_00199", "whucd_01396", "whucd_02606", "whucd_04108", "whucd_05272", "whucd_06460",
    "whucd_00209", "whucd_01420", "whucd_02608", "whucd_04118", "whucd_05278", "whucd_06466",
    "whucd_00222", "whucd_01425", "whucd_02609", "whucd_04127", "whucd_05280", "whucd_06482",
    "whucd_00225", "whucd_01442", "whucd_02621", "whucd_04132", "whucd_05284", "whucd_06486",
    "whucd_00226", "whucd_01447", "whucd_02631", "whucd_04140", "whucd_05285", "whucd_06494",
    "whucd_00230", "whucd_01472", "whucd_02641", "whucd_04142", "whucd_05292", "whucd_06496",
    "whucd_00234", "whucd_01500", "whucd_02661", "whucd_04145", "whucd_05293", "whucd_06497",
    "whucd_00240", "whucd_01510", "whucd_02666", "whucd_04146", "whucd_05312", "whucd_06507",
    "whucd_00264", "whucd_01515", "whucd_02671", "whucd_04155", "whucd_05316", "whucd_06509",
    "whucd_00272", "whucd_01518", "whucd_02679", "whucd_04160", "whucd_05319", "whucd_06511",
    "whucd_00275", "whucd_01519", "whucd_02702", "whucd_04163", "whucd_05324", "whucd_06519",
    "whucd_00277", "whucd_01542", "whucd_02734", "whucd_04184", "whucd_05326", "whucd_06522",
    "whucd_00293", "whucd_01543", "whucd_02750", "whucd_04185", "whucd_05341", "whucd_06525",
    "whucd_00311", "whucd_01564", "whucd_02757", "whucd_04191", "whucd_05359", "whucd_06540",
    "whucd_00320", "whucd_01566", "whucd_02765", "whucd_04196", "whucd_05361", "whucd_06543",
    "whucd_00326", "whucd_01567", "whucd_02766", "whucd_04198", "whucd_05378", "whucd_06549",
    "whucd_00329", "whucd_01568", "whucd_02793", "whucd_04200", "whucd_05395", "whucd_06560",
    "whucd_00337", "whucd_01570", "whucd_02797", "whucd_04209", "whucd_05398", "whucd_06584",
    "whucd_00348", "whucd_01599", "whucd_02810", "whucd_04255", "whucd_05406", "whucd_06587",
    "whucd_00349", "whucd_01606", "whucd_02842", "whucd_04265", "whucd_05408", "whucd_06594",
    "whucd_00365", "whucd_01613", "whucd_02853", "whucd_04269", "whucd_05409", "whucd_06601",
    "whucd_00368", "whucd_01615", "whucd_02891", "whucd_04273", "whucd_05438", "whucd_06607",
    "whucd_00373", "whucd_01624", "whucd_02933", "whucd_04292", "whucd_05455", "whucd_06645",
    "whucd_00385", "whucd_01634", "whucd_02976", "whucd_04333", "whucd_05465", "whucd_06651",
    "whucd_00403", "whucd_01645", "whucd_03019", "whucd_04335", "whucd_05466", "whucd_06669",
    "whucd_00416", "whucd_01647", "whucd_03031", "whucd_04349", "whucd_05471", "whucd_06670",
    "whucd_00417", "whucd_01648", "whucd_03036", "whucd_04360", "whucd_05475", "whucd_06695",
    "whucd_00419", "whucd_01682", "whucd_03037", "whucd_04367", "whucd_05482", "whucd_06712",
    "whucd_00424", "whucd_01724", "whucd_03054", "whucd_04376", "whucd_05488", "whucd_06715",
    "whucd_00446", "whucd_01736", "whucd_03061", "whucd_04406", "whucd_05493", "whucd_06720",
    "whucd_00447", "whucd_01759", "whucd_03068", "whucd_04434", "whucd_05497", "whucd_06722",
    "whucd_00460", "whucd_01781", "whucd_03085", "whucd_04436", "whucd_05503", "whucd_06736",
    "whucd_00465", "whucd_01797", "whucd_03089", "whucd_04448", "whucd_05509", "whucd_06759",
    "whucd_00471", "whucd_01800", "whucd_03105", "whucd_04459", "whucd_05546", "whucd_06777",
    "whucd_00477", "whucd_01806", "whucd_03109", "whucd_04466", "whucd_05561", "whucd_06798",
    "whucd_00480", "whucd_01810", "whucd_03122", "whucd_04470", "whucd_05569", "whucd_06823",
    "whucd_00491", "whucd_01812", "whucd_03128", "whucd_04478", "whucd_05587", "whucd_06866",
    "whucd_00506", "whucd_01814", "whucd_03169", "whucd_04482", "whucd_05588", "whucd_06879",
    "whucd_00519", "whucd_01818", "whucd_03171", "whucd_04489", "whucd_05596", "whucd_06885",
    "whucd_00522", "whucd_01855", "whucd_03188", "whucd_04491", "whucd_05599", "whucd_06907",
    "whucd_00551", "whucd_01871", "whucd_03193", "whucd_04492", "whucd_05604", "whucd_06930",
    "whucd_00559", "whucd_01878", "whucd_03202", "whucd_04497", "whucd_05609", "whucd_06938",
    "whucd_00563", "whucd_01882", "whucd_03208", "whucd_04506", "whucd_05627", "whucd_06939",
    "whucd_00579", "whucd_01883", "whucd_03210", "whucd_04527", "whucd_05628", "whucd_06952",
    "whucd_00580", "whucd_01886", "whucd_03230", "whucd_04538", "whucd_05632", "whucd_06966",
    "whucd_00582", "whucd_01891", "whucd_03231", "whucd_04539", "whucd_05634", "whucd_06970",
    "whucd_00595", "whucd_01897", "whucd_03244", "whucd_04546", "whucd_05636", "whucd_06975",
    "whucd_00600", "whucd_01900", "whucd_03250", "whucd_04548", "whucd_05640", "whucd_06998",
    "whucd_00612", "whucd_01905", "whucd_03255", "whucd_04558", "whucd_05643", "whucd_07008",
    "whucd_00648", "whucd_01919", "whucd_03257", "whucd_04613", "whucd_05646", "whucd_07009",
    "whucd_00658", "whucd_01920", "whucd_03295", "whucd_04620", "whucd_05647", "whucd_07016",
    "whucd_00660", "whucd_01952", "whucd_03298", "whucd_04624", "whucd_05659", "whucd_07020",
    "whucd_00662", "whucd_01971", "whucd_03299", "whucd_04634", "whucd_05667", "whucd_07034",
    "whucd_00668", "whucd_01973", "whucd_03302", "whucd_04637", "whucd_05668", "whucd_07051",
    "whucd_00671", "whucd_01998", "whucd_03322", "whucd_04652", "whucd_05713", "whucd_07073",
    "whucd_00684", "whucd_02001", "whucd_03353", "whucd_04684", "whucd_05727", "whucd_07077",
    "whucd_00685", "whucd_02003", "whucd_03357", "whucd_04687", "whucd_05729", "whucd_07079",
    "whucd_00686", "whucd_02005", "whucd_03380", "whucd_04709", "whucd_05742", "whucd_07081",
    "whucd_00690", "whucd_02014", "whucd_03388", "whucd_04718", "whucd_05754", "whucd_07089",
    "whucd_00696", "whucd_02016", "whucd_03400", "whucd_04719", "whucd_05764", "whucd_07093",
    "whucd_00698", "whucd_02024", "whucd_03401", "whucd_04733", "whucd_05772", "whucd_07102",
    "whucd_00701", "whucd_02046", "whucd_03404", "whucd_04744", "whucd_05777", "whucd_07119",
    "whucd_00711", "whucd_02047", "whucd_03412", "whucd_04765", "whucd_05797", "whucd_07121",
    "whucd_00731", "whucd_02053", "whucd_03418", "whucd_04766", "whucd_05824", "whucd_07132",
    "whucd_00735", "whucd_02056", "whucd_03429", "whucd_04773", "whucd_05825", "whucd_07141",
    "whucd_00756", "whucd_02060", "whucd_03435", "whucd_04780", "whucd_05847", "whucd_07151",
    "whucd_00758", "whucd_02064", "whucd_03437", "whucd_04785", "whucd_05858", "whucd_07164",
    "whucd_00776", "whucd_02065", "whucd_03457", "whucd_04787", "whucd_05883", "whucd_07165",
    "whucd_00777", "whucd_02077", "whucd_03458", "whucd_04797", "whucd_05893", "whucd_07168",
    "whucd_00786", "whucd_02086", "whucd_03477", "whucd_04805", "whucd_05901", "whucd_07171",
    "whucd_00810", "whucd_02109", "whucd_03481", "whucd_04810", "whucd_05927", "whucd_07189",
    "whucd_00811", "whucd_02117", "whucd_03510", "whucd_04824", "whucd_05948", "whucd_07190",
    "whucd_00814", "whucd_02146", "whucd_03515", "whucd_04825", "whucd_05951", "whucd_07197",
    "whucd_00825", "whucd_02152", "whucd_03515"
]


for img in images:
    annotate_image(img)

TypeError: annotate_image() missing 1 required positional argument: 'dataset'

# LEVIRCD

In [ ]:
Images = [
    "00638", "00639", "00640", "00641", "00642", "00643", "00644", "00645", "00646", "00647", "00648", "00649", "00650", "00651", "00652", "00653",
    "00654", "00655", "00656", "00657", "00658", "00659", "00660", "00661", "00662", "00663", "00664", "00665", "00666", "00667", "00668", "00669",
    "00670", "00671", "00672", "00673", "00674", "00675", "00676", "00677", "00678", "00679", "00680", "00681", "00682", "00683", "00684", "00685",
    "00686", "00687", "00688", "00689", "00690", "00691", "00692", "00693", "00694", "00695", "00696", "00697", "00698", "00699", "00700", "00701",
    "00702", "00703", "00704", "00705", "00706", "00707", "00708", "00709", "00710", "00711", "00712", "00713", "00714", "00715", "00716", "00717",
    "00718", "00719", "00720", "00721", "00722", "00723", "00724", "00725", "00726", "00727", "00728", "00729", "00730", "00731", "00732", "00733",
    "00734", "00735", "00736", "00737", "00738", "00739", "00740", "00741", "00742", "00743", "00744", "00745", "00746", "00747", "00748", "00749",
    "00750", "00751", "00752", "00753", "00754", "00755", "00756", "00757", "00758", "00759", "00760", "00761", "00762", "00763", "00764", "00765",
    "00766", "00767", "00768", "00769", "00770", "00771", "00772", "00773", "00774", "00775", "00776", "00777", "00778", "00779", "00780", "00781",
    "00782", "00783", "00784", "00785", "00786", "00787", "00788", "00789", "00790", "00791", "00792", "00793", "00794", "00795", "00796", "00797",
    "00798", "00799", "00800", "00801", "00802", "00803", "00804", "00805", "00806", "00807", "00808", "00809", "00810", "00811", "00812", "00813",
    "00814", "00815", "00816", "00817", "00818", "00819", "00820", "00821", "00822", "00823", "00824", "00825", "00826", "00827", "00828", "00829",
    "00830", "00831", "00832", "00833", "00834", "00835", "00836", "00837", "00838", "00839", "00840", "00841", "00842", "00843", "00844", "00845",
    "00846", "00847", "00848", "00849", "00850", "00851", "00852", "00853", "00854", "00855", "00856", "00857", "00858", "00859", "00860", "00861",
    "00862", "00863", "00864", "00865", "00866", "00867", "00868", "00869", "00870", "00871", "00872", "00873", "00874", "00875", "00876", "00877",
    "00878", "00879", "00880", "00881", "00882", "00883", "00884", "00885", "00886", "00887", "00888", "00889", "00890", "00891", "00892", "00893",
    "00894", "00895", "00896", "00897", "00898", "00899", "00900", "00901", "00902", "00903", "00904", "00905", "00906", "00907", "00908", "00909",
    "00910", "00911", "00912", "00913", "00914", "00915", "00916", "00917", "00918", "00919", "00920", "00921", "00922", "00923", "00924", "00925",
    "00926", "00927", "00928", "00929", "00930", "00931", "00932", "00933", "00934", "00935", "00936", "00937", "00938", "00939", "00940", "00941",
    "00942", "00943", "00944", "00945", "00946", "00947", "00948", "00949", "00950", "00951", "00952", "00953", "00954", "00955", "00956", "00957",
    "00958", "00959", "00960", "00961", "00962", "00963", "00964", "00965", "00966", "00967", "00968", "00969", "00970", "00971", "00972", "00973",
    "00974", "00975", "00976", "00977", "00978", "00979", "00980", "00981", "00982", "00983", "00984", "00985"
]

for img in Images:
    annotate_image(img, "LEVIR-CD+")

/tmp/ipykernel_209599/3105635309.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = np.array(imageio.imread(path), np.float32)


# SYSU

In [25]:
Images = [
     "00638", "00639", "00640", "00641", "00642", "00643", "00644", "00645", "00646", "00647",
    "00648", "00649", "00650", "00651", "00652", "00653", "00654", "00655", "00656", "00657",
    "00658", "00659", "00660", "00661", "00662", "00663", "00664", "00665", "00666", "00667",
    "00668", "00669", "00670", "00671", "00672", "00673", "00674", "00675", "00676", "00677",
    "00678", "00679", "00680", "00681", "00682", "00683", "00684", "00685", "00686", "00687",
    "00688", "00689", "00690", "00691", "00692", "00693", "00694", "00695", "00696", "00697",
    "00698", "00699", "00700", "00701", "00702", "00703", "00704", "00705", "00706", "00707",
    "00708", "00709", "00710", "00711", "00712", "00713", "00714", "00715", "00716", "00717",
    "00718", "00719", "00720", "00721", "00722", "00723", "00724", "00725", "00726", "00727",
    "00728", "00729", "00730", "00731", "00732", "00733", "00734", "00735", "00736", "00737",
    "00738", "00739", "00740", "00741", "00742", "00743", "00744", "00745", "00746", "00747",
    "00748", "00749", "00750", "00751", "00752", "00753", "00754", "00755", "00756", "00757",
    "00758", "00759", "00760", "00761", "00762", "00763", "00764", "00765", "00766", "00767",
    "00768", "00769", "00770", "00771", "00772", "00773", "00774", "00775", "00776", "00777",
    "00778", "00779", "00780", "00781", "00782", "00783", "00784", "00785", "00786", "00787",
    "00788", "00789", "00790", "00791", "00792", "00793", "00794", "00795", "00796", "00797",
    "00798", "00799", "00800", "00801", "00802", "00803", "00804", "00805", "00806", "00807",
    "00808", "00809", "00810", "00811", "00812", "00813", "00814", "00815", "00816", "00817",
    "00818", "00819", "00820", "00821", "00822", "00823", "00824", "00825", "00826", "00827",
    "00828", "00829", "00830", "00831", "00832", "00833", "00834", "00835", "00836", "00837",
    "00838", "00839", "00840", "00841", "00842", "00843", "00844", "00845", "00846", "00847",
    "00848", "00849", "00850", "00851", "00852", "00853", "00854", "00855", "00856", "00857",
    "00858", "00859", "00860", "00861", "00862", "00863", "00864", "00865", "00866", "00867",
    "00868", "00869", "00870", "00871", "00872", "00873", "00874", "00875", "00876", "00877",
    "00878", "00879", "00880", "00881", "00882", "00883", "00884", "00885", "00886", "00887",
    "00888", "00889", "00890", "00891", "00892", "00893", "00894", "00895", "00896", "00897",
    "00898", "00899", "00900", "00901", "00902", "00903", "00904", "00905", "00906", "00907",
    "00908", "00909", "00910", "00911", "00912", "00913", "00914", "00915", "00916", "00917",
    "00918", "00919", "00920", "00921", "00922", "00923", "00924", "00925", "00926", "00927",
    "00928", "00929", "00930", "00931", "00932", "00933", "00934", "00935", "00936", "00937",
    "00938", "00939", "00940", "00941", "00942", "00943", "00944", "00945", "00946", "00947"
]

for img in Images:
    annotate_image(img, "SYSU")

/tmp/ipykernel_209599/3105635309.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = np.array(imageio.imread(path), np.float32)
